In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [4]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [5]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [9]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [10]:
model = VAE().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and KL divergence
        # For KL divergence, ssee Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Reconst Loss: {reconst_loss.item():.4f}, KL Div: {kl_div.item():.4f}')
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, f'sampled-{epoch+1}.png'))
        
        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, f'reconst-{epoch+1}.png'))

/root/pytorch-tutorial/.venv/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch [1/15], Step [10/469], Reconst Loss: 37033.7617, KL Div: 2695.2554
Epoch [1/15], Step [20/469], Reconst Loss: 29164.4336, KL Div: 1190.9150
Epoch [1/15], Step [30/469], Reconst Loss: 27791.5879, KL Div: 1207.1854
Epoch [1/15], Step [40/469], Reconst Loss: 26143.7480, KL Div: 704.9409
Epoch [1/15], Step [50/469], Reconst Loss: 27443.6875, KL Div: 643.5421
Epoch [1/15], Step [60/469], Reconst Loss: 25088.0723, KL Div: 894.3195
Epoch [1/15], Step [70/469], Reconst Loss: 25697.2734, KL Div: 739.8616
Epoch [1/15], Step [80/469], Reconst Loss: 25151.4570, KL Div: 804.2756
Epoch [1/15], Step [90/469], Reconst Loss: 23585.9629, KL Div: 1045.5264
Epoch [1/15], Step [100/469], Reconst Loss: 22869.0859, KL Div: 1368.6306
Epoch [1/15], Step [110/469], Reconst Loss: 21414.1758, KL Div: 1531.9656
Epoch [1/15], Step [120/469], Reconst Loss: 20448.5547, KL Div: 1427.9634
Epoch [1/15], Step [130/469], Reconst Loss: 20836.2793, KL Div: 1578.3391
Epoch [1/15], Step [140/469], Reconst Loss: 20145.51